### Summary

The idea here is to design a dnn prediction algorithm using tensorflow. We will be using estimator API, which allows us seamless transition to other models !

In [22]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import shutil

In [23]:
# The directory for storing metadata related to the model. This way, we can easily build models step by step, starting
# from the previous checkpoint, in cases when we have large amount of data.
OUTDIR = 'sample_model_metadata'

In [ ]:
#INPUT_DIR = '../input/'
INPUT_DIR = '../input/competitive-data-science-predict-future-sales/'

In [24]:
#  The pandas input function which is to be fed to the estimator API. NOTE : Since the syntax is very similar, we could 
# have used one function for training and evaluation as well, but we have decided to keep them separately for the sake
# of clarity.
def make_train_input_fn(df, num_epochs=1, shuffle=True):
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        y=df['item_cnt_month'],
        batch_size=128,
        num_epochs=num_epochs,
        shuffle=shuffle,
        queue_capacity=2000)

def make_eval_input_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        y=df['item_cnt_month'],
        batch_size=128,
        num_epochs=1,
        shuffle=False,
        queue_capacity=2000)

def make_prediction_input_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        batch_size=128,
        num_epochs=1,
        shuffle=False,
        queue_capacity=2000)

In [25]:
REL_PREDICTOR_COLS = ['date_block_num', 'shop_id', 'item_id', 'item_category_id']

In [26]:
# This routine makes the input in a tensorflow digestible form.
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in REL_PREDICTOR_COLS]
  return input_columns


##### Get relevant data into pandas dataframes.

In [27]:
sales_train = pd.read_csv(INPUT_DIR + "sales_train.csv")

In [28]:
items = pd.read_csv(INPUT_DIR + 'items.csv')

In [29]:
item_id_to_item_category_id = dict(zip(items.item_id, items.item_category_id))

In [30]:
monthly_sales_data = sales_train[['date_block_num', 'shop_id', 'item_id', 'item_cnt_day' ]].groupby(
    ['date_block_num', 'shop_id', 'item_id']).sum()
monthly_sales_data.rename(columns={'item_cnt_day':'item_cnt_month'}, inplace=True)
monthly_sales_data.reset_index(['date_block_num', 'shop_id', 'item_id'], inplace=True)

In [31]:
monthly_sales_data['item_category_id'] = monthly_sales_data['item_id'].apply(lambda x : item_id_to_item_category_id.get(x, -1))

In [32]:
monthly_sales_data.columns

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'item_category_id'],
      dtype='object')

In [33]:
train_data = monthly_sales_data[(monthly_sales_data.date_block_num != 32) &(monthly_sales_data.date_block_num != 33)]
validation_data = monthly_sales_data[monthly_sales_data.date_block_num == 32]
test_data = monthly_sales_data[monthly_sales_data.date_block_num == 33]

##### Addtitional specific tensorflow functions.

In [34]:
# Train any model, for which a tensorflow estimator API is built. This way, we can test out several models with a one
# line change.
def get_trained_model(model, train_data, outdir=OUTDIR, logging_mode=tf.logging.INFO):
    
    tf.logging.set_verbosity(logging_mode)
    
    # Delete the directory corresponding to metadata of model so as to build it from scratch.
    shutil.rmtree(outdir, ignore_errors=True)

    # Train data for a reasonable number of epochs (100) as default.
    model.train(make_train_input_fn(train_data, num_epochs=100, shuffle=False))
    
    return model

In [35]:
def get_predictions_from_model(model, test_data):
    predictions = model.predict(input_fn=make_prediction_input_fn(test_data)) 
    return [x['predictions'][0] for x in predictions]

##### Train the model and obtain validation score. Also, make score that the predictions are clipped so that they make sense

In [ ]:
HIDDEN_UNITS = [20, 20]
#HIDDEN_UNITS = [10, 10, 10]
#HIDDEN_UNITS = [10, 10]
#HIDDEN_UNITS = [5, 5]
#HIDDEN_UNITS = [10]
#HIDDEN_UNITS = [3]
model = get_trained_model(tf.estimator.DNNRegressor(feature_columns=make_feature_cols(),
                                                    hidden_units=HIDDEN_UNITS,
                                                    model_dir=OUTDIR),
                         train_data)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'sample_model_metadata', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x152e31198>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `t

INFO:tensorflow:global_step/sec: 144.878
INFO:tensorflow:loss = 3325.0034, step = 5501 (0.690 sec)
INFO:tensorflow:global_step/sec: 169.367
INFO:tensorflow:loss = 1474.8767, step = 5601 (0.590 sec)
INFO:tensorflow:global_step/sec: 151.387
INFO:tensorflow:loss = 123.59081, step = 5701 (0.661 sec)
INFO:tensorflow:global_step/sec: 184.005
INFO:tensorflow:loss = 129.19156, step = 5801 (0.543 sec)
INFO:tensorflow:global_step/sec: 178.394
INFO:tensorflow:loss = 482.0095, step = 5901 (0.561 sec)
INFO:tensorflow:global_step/sec: 147.907
INFO:tensorflow:loss = 436.64053, step = 6001 (0.676 sec)
INFO:tensorflow:global_step/sec: 176.316
INFO:tensorflow:loss = 17771.377, step = 6101 (0.567 sec)
INFO:tensorflow:global_step/sec: 177.839
INFO:tensorflow:loss = 606.74817, step = 6201 (0.562 sec)
INFO:tensorflow:global_step/sec: 152.89
INFO:tensorflow:loss = 2962.8152, step = 6301 (0.654 sec)
INFO:tensorflow:global_step/sec: 177.083
INFO:tensorflow:loss = 19584.566, step = 6401 (0.565 sec)
INFO:tensorf

INFO:tensorflow:global_step/sec: 171.32
INFO:tensorflow:loss = 1024.4998, step = 13801 (0.584 sec)
INFO:tensorflow:global_step/sec: 146.205
INFO:tensorflow:loss = 143.18312, step = 13901 (0.684 sec)
INFO:tensorflow:global_step/sec: 161.659
INFO:tensorflow:loss = 573.54297, step = 14001 (0.619 sec)
INFO:tensorflow:global_step/sec: 174.818
INFO:tensorflow:loss = 895.7832, step = 14101 (0.572 sec)
INFO:tensorflow:global_step/sec: 128.442
INFO:tensorflow:loss = 456.96085, step = 14201 (0.778 sec)
INFO:tensorflow:global_step/sec: 167.574
INFO:tensorflow:loss = 3393.2153, step = 14301 (0.597 sec)
INFO:tensorflow:global_step/sec: 135.74
INFO:tensorflow:loss = 910.4501, step = 14401 (0.737 sec)
INFO:tensorflow:global_step/sec: 158.709
INFO:tensorflow:loss = 395.55432, step = 14501 (0.630 sec)
INFO:tensorflow:global_step/sec: 160.676
INFO:tensorflow:loss = 265.75104, step = 14601 (0.622 sec)
INFO:tensorflow:global_step/sec: 142.684
INFO:tensorflow:loss = 4860.828, step = 14701 (0.701 sec)
INFO:

INFO:tensorflow:loss = 424.659, step = 22001 (0.703 sec)
INFO:tensorflow:global_step/sec: 155.968
INFO:tensorflow:loss = 1249.8943, step = 22101 (0.641 sec)
INFO:tensorflow:global_step/sec: 132.954
INFO:tensorflow:loss = 317.14236, step = 22201 (0.752 sec)
INFO:tensorflow:global_step/sec: 163.848
INFO:tensorflow:loss = 1154.4087, step = 22301 (0.610 sec)
INFO:tensorflow:global_step/sec: 166.573
INFO:tensorflow:loss = 4221.5205, step = 22401 (0.600 sec)
INFO:tensorflow:global_step/sec: 131.771
INFO:tensorflow:loss = 1907.5615, step = 22501 (0.759 sec)
INFO:tensorflow:global_step/sec: 168.24
INFO:tensorflow:loss = 502.76093, step = 22601 (0.594 sec)
INFO:tensorflow:global_step/sec: 141.032
INFO:tensorflow:loss = 428.7353, step = 22701 (0.709 sec)
INFO:tensorflow:global_step/sec: 155.908
INFO:tensorflow:loss = 71618.72, step = 22801 (0.642 sec)
INFO:tensorflow:global_step/sec: 166.358
INFO:tensorflow:loss = 148.54704, step = 22901 (0.601 sec)
INFO:tensorflow:global_step/sec: 150.312
INFO:

INFO:tensorflow:global_step/sec: 175.91
INFO:tensorflow:loss = 2947.052, step = 30301 (0.568 sec)
INFO:tensorflow:global_step/sec: 150.916
INFO:tensorflow:loss = 4031.3152, step = 30401 (0.663 sec)
INFO:tensorflow:global_step/sec: 177.563
INFO:tensorflow:loss = 1245.9366, step = 30501 (0.563 sec)
INFO:tensorflow:global_step/sec: 149.723
INFO:tensorflow:loss = 37149.43, step = 30601 (0.668 sec)
INFO:tensorflow:global_step/sec: 180.286
INFO:tensorflow:loss = 9599.261, step = 30701 (0.555 sec)
INFO:tensorflow:global_step/sec: 176.723
INFO:tensorflow:loss = 1607.2871, step = 30801 (0.566 sec)
INFO:tensorflow:global_step/sec: 131.819
INFO:tensorflow:loss = 953.22565, step = 30901 (0.759 sec)
INFO:tensorflow:global_step/sec: 169.809
INFO:tensorflow:loss = 585.37286, step = 31001 (0.589 sec)
INFO:tensorflow:global_step/sec: 145.198
INFO:tensorflow:loss = 3443.0608, step = 31101 (0.689 sec)
INFO:tensorflow:global_step/sec: 177.811
INFO:tensorflow:loss = 2649.145, step = 31201 (0.562 sec)
INFO:

...

INFO:tensorflow:global_step/sec: 169.342
INFO:tensorflow:loss = 811.6966, step = 194601 (0.591 sec)
INFO:tensorflow:global_step/sec: 140.661
INFO:tensorflow:loss = 1851.0186, step = 194701 (0.711 sec)
INFO:tensorflow:global_step/sec: 160.007
INFO:tensorflow:loss = 1266.2595, step = 194801 (0.625 sec)
INFO:tensorflow:global_step/sec: 162.264
INFO:tensorflow:loss = 276.59424, step = 194901 (0.616 sec)
INFO:tensorflow:global_step/sec: 139.556
INFO:tensorflow:loss = 2522.454, step = 195001 (0.717 sec)
INFO:tensorflow:global_step/sec: 153.639
INFO:tensorflow:loss = 17716.512, step = 195101 (0.651 sec)
INFO:tensorflow:global_step/sec: 138.085
INFO:tensorflow:loss = 1101.7301, step = 195201 (0.724 sec)
INFO:tensorflow:global_step/sec: 163.895
INFO:tensorflow:loss = 2171.7788, step = 195301 (0.610 sec)
INFO:tensorflow:global_step/sec: 159.981
INFO:tensorflow:loss = 1091.2957, step = 195401 (0.626 sec)
INFO:tensorflow:global_step/sec: 144.234
INFO:tensorflow:loss = 173.12047, step = 195501 (0.6

INFO:tensorflow:global_step/sec: 146.665
INFO:tensorflow:loss = 477.9585, step = 202801 (0.682 sec)
INFO:tensorflow:global_step/sec: 172.533
INFO:tensorflow:loss = 290.11404, step = 202901 (0.580 sec)
INFO:tensorflow:global_step/sec: 173.94
INFO:tensorflow:loss = 299.02304, step = 203001 (0.575 sec)
INFO:tensorflow:global_step/sec: 140.103
INFO:tensorflow:loss = 264.30664, step = 203101 (0.714 sec)
INFO:tensorflow:global_step/sec: 165.83
INFO:tensorflow:loss = 606.0371, step = 203201 (0.603 sec)
INFO:tensorflow:global_step/sec: 151.143
INFO:tensorflow:loss = 436.76312, step = 203301 (0.662 sec)
INFO:tensorflow:global_step/sec: 173.38
INFO:tensorflow:loss = 383.09637, step = 203401 (0.577 sec)
INFO:tensorflow:global_step/sec: 169.584
INFO:tensorflow:loss = 175.38466, step = 203501 (0.589 sec)
INFO:tensorflow:global_step/sec: 142.671
INFO:tensorflow:loss = 31062.627, step = 203601 (0.701 sec)
INFO:tensorflow:global_step/sec: 176.207
INFO:tensorflow:loss = 845.21454, step = 203701 (0.567 

INFO:tensorflow:global_step/sec: 148.399
INFO:tensorflow:loss = 895.8341, step = 211001 (0.674 sec)
INFO:tensorflow:global_step/sec: 175.232
INFO:tensorflow:loss = 333.60114, step = 211101 (0.570 sec)
INFO:tensorflow:global_step/sec: 147.209
INFO:tensorflow:loss = 1091.6581, step = 211201 (0.679 sec)
INFO:tensorflow:global_step/sec: 176.356
INFO:tensorflow:loss = 268.31073, step = 211301 (0.567 sec)
INFO:tensorflow:global_step/sec: 174.308
INFO:tensorflow:loss = 970.06775, step = 211401 (0.574 sec)
INFO:tensorflow:global_step/sec: 145.053
INFO:tensorflow:loss = 742.96045, step = 211501 (0.689 sec)
INFO:tensorflow:global_step/sec: 171.886
INFO:tensorflow:loss = 717.5641, step = 211601 (0.582 sec)
INFO:tensorflow:global_step/sec: 174.293
INFO:tensorflow:loss = 518.3102, step = 211701 (0.574 sec)
INFO:tensorflow:global_step/sec: 139.026
INFO:tensorflow:loss = 251.20917, step = 211801 (0.719 sec)
INFO:tensorflow:global_step/sec: 149.478
INFO:tensorflow:loss = 382.41208, step = 211901 (0.66

INFO:tensorflow:global_step/sec: 147.265
INFO:tensorflow:loss = 561.2367, step = 219201 (0.679 sec)
INFO:tensorflow:global_step/sec: 169.045
INFO:tensorflow:loss = 20684.828, step = 219301 (0.592 sec)
INFO:tensorflow:global_step/sec: 176.128
INFO:tensorflow:loss = 274.1902, step = 219401 (0.568 sec)
INFO:tensorflow:global_step/sec: 146.196
INFO:tensorflow:loss = 1831.936, step = 219501 (0.684 sec)
INFO:tensorflow:global_step/sec: 169.815
INFO:tensorflow:loss = 483.07935, step = 219601 (0.589 sec)
INFO:tensorflow:global_step/sec: 148.308
INFO:tensorflow:loss = 627.87476, step = 219701 (0.674 sec)
INFO:tensorflow:global_step/sec: 175.236
INFO:tensorflow:loss = 194.27878, step = 219801 (0.571 sec)
INFO:tensorflow:global_step/sec: 167.016
INFO:tensorflow:loss = 197767.67, step = 219901 (0.599 sec)
INFO:tensorflow:global_step/sec: 145.841
INFO:tensorflow:loss = 180.15106, step = 220001 (0.686 sec)
INFO:tensorflow:global_step/sec: 174.016
INFO:tensorflow:loss = 4938.1177, step = 220101 (0.57

INFO:tensorflow:global_step/sec: 175.806
INFO:tensorflow:loss = 13936.023, step = 227401 (0.569 sec)
INFO:tensorflow:global_step/sec: 139.569
INFO:tensorflow:loss = 79.88697, step = 227501 (0.716 sec)
INFO:tensorflow:global_step/sec: 175.772
INFO:tensorflow:loss = 3950.9758, step = 227601 (0.569 sec)
INFO:tensorflow:global_step/sec: 144.377
INFO:tensorflow:loss = 1403.7085, step = 227701 (0.693 sec)
INFO:tensorflow:global_step/sec: 175.432
INFO:tensorflow:loss = 713.10693, step = 227801 (0.570 sec)
INFO:tensorflow:global_step/sec: 171.352
INFO:tensorflow:loss = 2079.495, step = 227901 (0.584 sec)
INFO:tensorflow:global_step/sec: 146.63
INFO:tensorflow:loss = 2842.8142, step = 228001 (0.682 sec)
INFO:tensorflow:global_step/sec: 180.291
INFO:tensorflow:loss = 1623.1011, step = 228101 (0.555 sec)
INFO:tensorflow:global_step/sec: 161.636
INFO:tensorflow:loss = 739.1556, step = 228201 (0.619 sec)
INFO:tensorflow:global_step/sec: 170.876
INFO:tensorflow:loss = 330.34216, step = 228301 (0.585

In [191]:
model_predictions = get_predictions_from_model(model, validation_data)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from sample_model_metadata/model.ckpt-1200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [192]:
def get_rmse(model_predictions, validation_data):
    actual_vals = np.clip(validation_data.item_cnt_month.values, 0, 20)
    return np.sqrt(mean_squared_error(actual_vals, np.clip(model_predictions, 0, 20)))

In [193]:
get_rmse(model_predictions, validation_data)

2.6254074716990776